In [1]:
import openai
import pandas as pd

word_dictionary = pd.read_csv("../data/processed/word_dictionary_with_definitions.csv", sep=',', encoding='cp949')
OPEN_API_KEY = ''

openai.api_key = OPEN_API_KEY

def get_completion(definition, word, level):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "[역할] 당신은 초등학교 국어 선생님입니다. 해당 단어의 사전적 정보를 확인한 후 유의어 혹은 반의어 유형의 4지선다 문제를 출제하세요. 적절한 유의어 혹은 반의어가 없는 경우 응답하지 않고 넘어가세요"},
                {"role": "user", "content": f"[방법]"
                                            f" 아래 주어진 지문과 정해진 질문 내용을 바탕으로 {level} 수준의 4지선다 형태의 객관식 문제 1개를 출제하세요. 단어의 사전적 정보를 확인하고 단어의 의미를 활용한 문장을 구성한 후 유의어 혹은 반의어 유형의 문제를 반드시 아래 Output의 유형에 맞는 구성으로 만드세요. 요구한 결과물만 출력해줘"
                                            f"[Input] "
                                            f"[단어]"
                                            f"{word}"
                                            f"[사전적 정보]  "
                                            f"{definition} "
                                            f"[난이도] - {level}"
                                            f""
                                            f"[Output]"
                                            f"{example}"}
            ],
            model="gpt-4o-mini"
        )
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

example = """
{
    "question": "다음 문장의 괄호친 단어와 유의어를 고르시오",
    "sentence" : "(심심한) 사과를 드립니다", 
    "options":  {
        "1":"지루하고 재미없는",
        "2":"마음을 표현하는 정도가 매우 깊음",
        "3":"미안하지 않은",
        "4":"음식의 간이 약한"
    },
    "answer": 2,
    "reason": "답에 대한 이유"
}
"""

responses = []
level_mapping = {
    1: "미취학아동",
    2: "초등학교 1학년",
    3: "초등학교 2학년",
    4: "초등학교 3학년",
    5: "초등학교 4학년",
    6: "초등학교 5학년",
    7: "초등학교 6학년"
}

for i, row in word_dictionary.iterrows():
    if (i+1)%50==0:
        print(i+1)
    education_level = level_mapping.get(row['Level'], "Unknown")
    response = get_completion(row['definition'], row['Word'], education_level)
    
    if response:
        # print(response)
        responses.append(response)
    else:
        responses.append("Failed to get a response from the API.")
        
# print(responses)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100


In [2]:
responses_df = pd.DataFrame(word_dictionary[['Level']])
responses_df['response'] = responses

responses_df.to_csv("../data/processed/dt01_12.csv", index=False, encoding='utf-8-sig')

In [8]:
import json
from datetime import datetime

def transform_row(row):
    response_json = json.loads(row['response'])
    options = response_json.get('options', {})
    
    answer_key = str(response_json.get("answer", ""))  
    answer_value = ""
    for k, v in options.items():
        if str(k) == answer_key:
            answer_value = v  
            break

    now = datetime.now().isoformat()
    
    transformed = {
        "title": response_json.get("question", ""),
        "content": response_json.get("sentence", ""),
        "answer" : answer_value,
        "solution": response_json.get("reason", ""),
        "questionTypeCode": "QT01",
        "problemLevelCode": f"PL0{row['Level']}",
        "problemTypeCode": "PT01",
        "problemTypeDetailCode": "PT0112",
        "metadata": {
            "options": [v for v in options.values()] 
        },
        "createdDate": now,
        "modifiedDate": now 
    }
    
    return transformed

transformed_data = []
for i, row in responses_df.iterrows():
    try:
        json.loads(row['response'])  # Try parsing the original JSON
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
        print(f"Problematic JSON: {row['response']}")

    
dt01_12_df = pd.DataFrame(transformed_data)

Error in row 568: Expecting property name enclosed in double quotes: line 9 column 5 (char 181)
Problematic JSON: {
    "question": "다음 문장의 괄호친 단어와 유의어를 고르시오",
    "sentence": "(재채기) 할 때 코가 간질거려요.",
    "options": {
        "1": "기침",
        "2": "눈물",
        "3": "콧물",
        "4": "숨",
    },
    "answer": 1,
    "reason": "재채기와 기침은 둘 다 호흡과 관련된 반응으로, 기침은 재채기의 유사한 표현으로 볼 수 있습니다."
}
Error in row 837: Expecting property name enclosed in double quotes: line 9 column 5 (char 183)
Problematic JSON: {
    "question": "다음 문장의 괄호친 단어와 유의어를 고르시오",
    "sentence" : "(협동)하여 문제를 해결했습니다",
    "options":  {
        "1":"함께 일하기",
        "2":"혼자서",
        "3":"각자",
        "4":"다툼",
    },
    "answer": 1,
    "reason": "협동은 함께 힘을 모아서 일하는 것을 의미하므로, '함께 일하기'가 유의어입니다."
}
Error in row 1017: Expecting property name enclosed in double quotes: line 9 column 5 (char 178)
Problematic JSON: {
    "question": "다음 문장의 괄호친 단어와 유의어를 고르시오",
    "sentence": "(도) 있는 사람은 놀아주세요.",
    "options": {
        "1": "껌"

In [9]:
print(dt01_12_df)
with open("../data/processed/problem_data/pt0112.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt01_12_df.to_csv("../data/processed/problem_data/pt0112.csv", index=False, encoding='utf-8-sig')

                            title                            content answer  \
0       다음 문장에서 괄호친 단어와 유의어를 고르시오                       나는 학교에 (가다).   이동하다   
1        다음 문장의 괄호친 단어와 유의어를 고르시오            (가방)은 물건을 넣어 다니는 용구입니다.     백팩   
2        다음 문장의 괄호친 단어와 유의어를 고르시오              그녀는 종이(가위)로 종이를 잘라냈다.     가위   
3       다음 문장에서 괄호친 단어와 유의어를 고르시오  (가위바위보)은 친구들과 함께 놀 때 자주 하는 게임이에요.     놀이   
4        다음 문장의 괄호친 단어와 유의어를 고르시오               나는 (가을)에는 단풍 구경을 간다.    가을철   
...                           ...                                ...    ...   
1114     다음 문장의 괄호친 단어와 유의어를 고르시오                 (처)와 함께 여행을 떠났습니다.     부인   
1115     다음 문장의 괄호친 단어와 유의어를 고르시오  그녀는 자신의 꿈을 (추구)하기 위해 열심히 노력하고 있다.     찾다   
1116  다음 문장에서 괄호 친 단어와 유의어를 고르시오.             (특성) 있는 사람을 만나면 매력적이다.     속성   
1117     다음 문장의 괄호친 단어와 유의어를 고르시오         그는 상황을 (판단)하여 적절한 결정을 내렸다.     평가   
1118    다음 문장에서 괄호친 단어의 유의어를 고르시오               이 방법은 좋은 (효과)를 거두었다.     결과   

                                               solu

In [11]:
dt01_12_df = pd.read_csv("../data/processed/problem_data/pt0112.csv", encoding='cp949')
dt01_12_df.to_csv("../data/processed/problem_data/pt0112.csv", index=False, encoding='utf-8-sig')
with open("../data/processed/problem_data/pt0112.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)